In [1]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [2]:
#df_All = pd.read_csv(r"C:\Users\saust\OneDrive - Sasol\1 Project rC4\Jupyter Notebooks\Report 10-20-23 No Fluff\df_All_Avg.csv")

df_All = pd.read_csv('df_Join_all.csv')


#df_All = pd.read_csv(r"C:\Users\saust\OneDrive\Desktop\CodeSpace DLs\102423\Project rC4\3 Final Machine Butanol\df_CDCA6.csv")


In [3]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [4]:
# # List of columns to exclude to run XGboost feature selection
# exclude_columns = ['Unnamed: 0', '%Al2O3_bM', 'Octanol PPM ', 'Hexanol PPM ',
#        'Ethanol PPM ', 'Dodecanol PPM ', 'Decanol PPM ', '%nC8OH', '%nC6OH', '%nC12OH', '%nC10OH ',
       
#        'TI52014', 'TI55013', 'TI55014', 'TI55015', 'TI55016', 'TI55017', 'TI55021', 'TI55023',
#        'TC52015', 'FC52018', 'II52554', 'TI40050', 'VI52558B'
#                    ]

# # Create a new DataFrame without the excluded columnsd
# df_All = df_All.drop(columns=exclude_columns)

In [5]:
# df_All = df_All[df_All['Date'] > '2022-06-15 00:00:00']

In [6]:

selected_columns = ['Butanol PPM',
                    '425 SAO Al', 'FFC55555', 'FC55003', 'FC55569',	
                    'TC55555', 'LC52572', 'LC55568', 'DI55152',	
                    'FC55009', 'FC55102',	
                    'LC55553', 'LC55557', 'LC90366'
                    ]

existing_columns = [col for col in selected_columns if col in df_All.columns]
df_All = df_All[existing_columns]

In [7]:
# First Iteration
# selected_columns = ['Butanol PPM', '425 SAO Al', 'FC42428', 'FC55003', 'FC55102', 'FC55152', 'FFC55555',
#        'LC55553', 'LC55557', 'LC55568', 'LC90366', 'TC55555']




In [8]:
df_All.columns

Index(['Butanol PPM', '425 SAO Al', 'FFC55555', 'FC55003', 'FC55569',
       'TC55555', 'LC52572', 'LC55568', 'DI55152', 'FC55009', 'FC55102',
       'LC55553', 'LC55557', 'LC90366'],
      dtype='object')

In [9]:
# Splitting into train and test
X = df_All.drop('Butanol PPM', axis=1)  # Assuming 'target' is your target column
y = df_All['Butanol PPM']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
#=============================================
# TRAIN & EVALUATE REGRESSION MODEL
#=============================================

#fit simple linear regression model
model = sm.OLS(y, X).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:            Butanol PPM   R-squared (uncentered):                   0.045
Model:                            OLS   Adj. R-squared (uncentered):              0.040
Method:                 Least Squares   F-statistic:                              9.949
Date:                Thu, 30 Nov 2023   Prob (F-statistic):                    9.70e-21
Time:                        08:05:11   Log-Likelihood:                         -21284.
No. Observations:                2770   AIC:                                  4.259e+04
Df Residuals:                    2757   BIC:                                  4.267e+04
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [11]:
# Convert summary to DataFrame
summary_table = model.summary().tables[1]
summary_df = pd.DataFrame(summary_table.data[1:], columns=summary_table.data[0])

# Convert necessary columns to numeric
numeric_cols = ['coef', 'std err', 't', 'P>|t|', '[0.025', '0.975]']
summary_df[numeric_cols] = summary_df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Apply the filter for insignificance and CI crossing zero
threshold = 0.05
filtered_df = summary_df[(summary_df['P>|t|'] > threshold) & ((summary_df['[0.025'] < 0) & (summary_df['0.975]'] > 0))]

# Extracting the feature names
insignificant_features = filtered_df.iloc[:, 0].tolist()  # First column contains feature names
print(insignificant_features)


['425 SAO Al', 'FC55569', 'TC55555', 'LC52572', 'LC55568', 'FC55009', 'FC55102', 'LC55553', 'LC55557', 'LC90366']


In [12]:
print(filtered_df)

                   coef  std err      t  P>|t|  [0.025  0.975]
0   425 SAO Al  13.5043   25.311  0.534  0.594 -36.126  63.135
3      FC55569   0.0238    0.032  0.748  0.455  -0.039   0.086
4      TC55555   1.4469    3.394  0.426  0.670  -5.208   8.102
5      LC52572   3.2490    5.089  0.638  0.523  -6.729  13.227
6      LC55568  10.8908    9.381  1.161  0.246  -7.503  29.285
8      FC55009   0.0175    0.019  0.904  0.366  -0.020   0.055
9      FC55102  -0.0003    0.002 -0.134  0.893  -0.004   0.003
10     LC55553  -2.7127    1.589 -1.708  0.088  -5.828   0.402
11     LC55557   1.5276    4.011  0.381  0.703  -6.338   9.393
12     LC90366  -0.7700    0.411 -1.875  0.061  -1.575   0.035


In [13]:
# Filter for significant features
significant_df = summary_df[(summary_df['P>|t|'] <= threshold) & ((summary_df['[0.025'] > 0) | (summary_df['0.975]'] < 0))]
significant_features = significant_df.iloc[:, 0].tolist()
print("\nSignificant Features:")
print(significant_features)


Significant Features:
['FFC55555', 'FC55003', 'DI55152']
